In [125]:
import pandas as pd
import spacy
nlp = spacy.load("pt_core_news_sm")

In [74]:
# from process_text import get_lemmas, tokenize_text, pos_tag_text

In [71]:
# corpus = pd.read_json('corpus.json')

In [ ]:
# tokenizado = corpus.apply(lambda x: tokenize_text(str(x['resumo'] + x['text']), 'portuguese'))

In [104]:
# post_tag = corpus.apply(lambda x: pos_tag_text(str(x['resumo']) + ' ' + str(x['text']), 'portuguese'), axis=1)

In [123]:
# lemmas = corpus.apply(lambda x: get_lemmas(str(x['resumo']) + ' ' + str(x['text']), 'portuguese'), axis=1)

In [122]:
# tokenizado

In [110]:
# corpus['lema'] = lemmas
# corpus['pos_tagger'] = post_tag
# corpus['artigo_tokenizado'] = tokenizado

In [124]:
# corpus.to_json('corpus.json', orient='records', lines=True)

In [113]:
corpus = pd.read_json('corpus.json', lines=True)

In [146]:
def extract_valid_tokens(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc
            if not token.is_stop and not token.is_punct and not token.is_space]

# Aplica a extração por linha
vocab = corpus['artigo_tokenizado'].apply(lambda x: extract_valid_tokens(str(x)))

# Flatten + removendo duplicados com set()
all_tokens = [token for tokens in vocab for token in tokens]
vocabulario = sorted(set(all_tokens))  # agora vai remover duplicados corretamente

print(f"Total de termos únicos no vocabulário: {len(vocabulario)}")

Total de termos únicos no vocabulário: 14938


In [150]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
import nltk
from nltk.corpus import stopwords
import string

# Baixar stopwords se necessário
nltk.download('stopwords')


# Combinar colunas de texto relevantes
corpus['texto_completo'] = corpus['resumo'] + ' ' + corpus['keywords'] + ' ' + corpus['text']

# Função de pré-processamento
def preprocess_text(text):
    if not isinstance(text, str):
        return ''

    # Converter para minúsculas
    text = text.lower()

    # Remover pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remover números
    text = ''.join([i for i in text if not i.isdigit()])

    # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))  # assumindo que o idioma é português
    words = text.split()
    words = [word for word in words if word not in stop_words and len(word) > 2]

    return ' '.join(words)

# Aplicar pré-processamento
corpus['texto_processado'] = corpus['texto_completo'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eduardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [157]:
# Criar vetorizador
vectorizer = CountVectorizer()

# Ajustar e transformar os dados
X = vectorizer.fit_transform(corpus['texto_processado'])

# Obter o vocabulário
vocabulario_bow = vectorizer.get_feature_names_out()

print(f"Tamanho do vocabulário BoW: {len(vocabulario_bow)}")
print("Exemplos de termos:", vocabulario_bow[:20])

Tamanho do vocabulário BoW: 14225
Exemplos de termos: ['aae' 'aaeexpr' 'aaes' 'aat' 'abaixo' 'abdômen' 'aberta' 'aberto'
 'abertos' 'abertura' 'abid' 'ablação' 'aborda' 'abordada' 'abordadas'
 'abordado' 'abordados' 'abordag' 'abordagem' 'abordagens']


In [167]:
# Criar vetorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()  # ajuste conforme necessário

# Ajustar e transformar os dados
X_tfidf = tfidf_vectorizer.fit_transform(corpus['texto_processado'])

# Obter o vocabulário
vocabulario_tfidf = tfidf_vectorizer.get_feature_names_out()

print(f"Tamanho do vocabulário TF-IDF: {len(vocabulario_tfidf)}")

Tamanho do vocabulário TF-IDF: 14225


In [170]:
# Criar vocabulário manual com base em todas as palavras
all_words = ' '.join(corpus['texto_processado']).split()
word_counts = Counter(all_words)

# Definir um mínimo de frequência
min_freq = 1  # ajuste conforme necessário
vocabulario_manual = [word for word, count in word_counts.items() if count >= min_freq]

print(f"Tamanho do vocabulário manual: {len(vocabulario_manual)}")

Tamanho do vocabulário manual: 14456
